# Liar Dataset

In [ ]:
import joblib
import pandas as pd

#Loading stuff
(linReg, weighted, undersampled) = joblib.load("../data/linReg.pkl")
vocab = joblib.load("../data/vectorizer.pkl")

column_names = ["id", "type", "text", "category", "person", "title", "state", 
                "party", "count1", "count2", "count3", "count4", "count5", "source"]

testDf = pd.read_csv("../data/liar/test.tsv", sep="\t", names=column_names)
print(testDf.head)
print(testDf.columns)
testDf.to_csv("../data/liar/ltest.csv", index=False)

<bound method NDFrame.head of               id         type  \
0     11972.json         true   
1     11685.json        false   
2     11096.json        false   
3      5209.json    half-true   
4      9524.json   pants-fire   
...          ...          ...   
1262   7334.json    half-true   
1263   9788.json  barely-true   
1264  10710.json  barely-true   
1265   3186.json  barely-true   
1266   6743.json        false   

                                                   text  \
0     Building a wall on the U.S.-Mexico border will...   
1     Wisconsin is on pace to double the number of l...   
2     Says John McCain has done nothing to help the ...   
3     Suzanne Bonamici supports a plan that will cut...   
4     When asked by a reporter whether hes at the ce...   
...                                                 ...   
1262  Says his budget provides the highest state fun...   
1263                    Ive been here almost every day.   
1264  In the early 1980s, Sen. Edward Kenn

In [ ]:
#To be run in the project.ipynb file
testDf = pd.read_csv("../data/liar/ltest.csv")
liar_processed = processData(testDf, 'text')
liar_processed.to_json(dataPath + "/liar/cleanLiar.json", orient="records", lines=True)


In [51]:

json_reader = pd.read_json('../data/liar/cleanLiar.json', orient='record', lines=True, chunksize=1500)
cleanLiar = pd.concat(json_reader, ignore_index=True)
cleanDf = cleanLiar.reset_index(drop=True) 
cleanDf.info()
print(cleanDf['type'].unique())

#Remove any where the type isn't true, false, or pants-fire
remove = ['half-true', 'mostly-true', 'barely-true']
cleanDf = cleanDf.loc[~testDf['type'].isin(remove)]
print(cleanDf['type'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1267 entries, 0 to 1266
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1267 non-null   object
 1   type      1267 non-null   object
 2   text      1267 non-null   object
 3   category  1267 non-null   object
 4   person    1267 non-null   object
 5   title     942 non-null    object
 6   state     1005 non-null   object
 7   party     1267 non-null   object
 8   count1    1267 non-null   int64 
 9   count2    1267 non-null   int64 
 10  count3    1267 non-null   int64 
 11  count4    1267 non-null   int64 
 12  count5    1267 non-null   int64 
 13  source    1250 non-null   object
dtypes: int64(5), object(9)
memory usage: 138.7+ KB
['true' 'false' 'half-true' 'pants-fire' 'barely-true' 'mostly-true']
['true' 'false' 'pants-fire']


In [52]:
#Replacing types with 1 and 0
cleanDf['type'] = cleanDf['type'].replace(r'^(true)$', '1', regex=True)
cleanDf['type'] = cleanDf['type'].replace(r'^(?!1$).+', '0', regex=True)
print(cleanDf['type'].unique())

['1' '0']


In [ ]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import CountVectorizer

liar_test = cleanDf['text']
liar_val = cleanDf['type'].astype(int)

vectorizer = CountVectorizer(analyzer=lambda x: x, vocabulary=vocab)
liar_Freq = vectorizer.fit_transform(liar_test)

scaler = Normalizer()
L_test = scaler.fit_transform(liar_Freq)

liar_pred = linReg.predict(L_test)
print(f"F1 score: {f1_score(liar_val, liar_pred):.4f}")
print(f"Accuracy: {accuracy_score(liar_val, liar_pred):.4f}")

F1 score: 0.3916
Accuracy: 0.5246
